In [1]:
# mnist cnn

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
input_width = 28
input_height = 28
input_depth = 1
input_pixels = 784

cnl1 = 32
cnl1_k = 5
cnl2 = 64
cnl2_k = 5
cnl1_stride = 1
cnl2_stride = 1
cnl1_maxpool = 2
cnl2_maxpool = 2

hidden_in = 1024
hidden_out = 10

input_size_to_hidden_layer =  (input_width*input_height//(cnl1_maxpool*cnl2_maxpool)**2) * cnl2 

In [4]:
input_size_to_hidden_layer

3136

In [20]:
weights = {
    "wc1":tf.Variable(tf.random_normal([cnl1_k,cnl1_k,input_depth,cnl1])),
    "wc2":tf.Variable(tf.random_normal([cnl2_k,cnl2_k,cnl1,cnl2])),
    "wh1":tf.Variable(tf.random_normal([input_size_to_hidden_layer,hidden_in])),
    "wo":tf.Variable(tf.random_normal([hidden_in,hidden_out]))
}

biases = {
    "bc1": tf.Variable(tf.random_normal([cnl1])),
    "bc2": tf.Variable(tf.random_normal([cnl2])),
    "bh1": tf.Variable(tf.random_normal([hidden_in])),
    "bo" : tf.Variable(tf.random_normal([hidden_out]))
}

In [21]:
def conv(x,weights,bias, strides = 1):
    out = tf.nn.conv2d(x,weights,padding = "SAME",strides = [1,strides,strides,1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding = "SAME",ksize = [1,k,k,1],strides = [1,k,k,1])
    

In [22]:
def cnn(x,weights,biases):
    x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_depth])
    o_cnl1 = conv(x,weights["wc1"],biases["bc1"],cnl1_stride)
    o_pool1 = maxpooling(o_cnl1 , cnl1_maxpool)
    
    o_cnl2 = conv(o_pool1 ,weights["wc2"],biases["bc2"],cnl2_stride)
    o_pool2 = maxpooling(o_cnl2,cnl2_maxpool)
    
    i_hidden = tf.reshape(o_pool2 , shape = [-1, input_size_to_hidden_layer ])
    o_hidden1 = tf.add(tf.matmul(i_hidden,weights["wh1"]),biases["bh1"])
    o_hidden1_activated = tf.nn.relu(o_hidden1)
    
    output = tf.add(tf.matmul(o_hidden1_activated,weights['wo']),biases["bo"])
    return output

In [23]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, hidden_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases)

In [24]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred,labels = y))

In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimizer = optimizer.minimize(cost) 

In [26]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [27]:
batch_size = 1000
for i in range(10):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run((cost,optimizer),feed_dict = {x:batch_x , y:batch_y})
        total_cost +=c
    print(total_cost)

903734.1203308105
13103.38565826416
6494.737899780273
3472.08097076416
1909.2965531349182
1106.9512255191803
579.4576704502106
317.1891052722931
182.61924697458744
108.43650814896682


In [28]:
predictions = tf.argmax(pred,1)
true_labels = tf.argmax(y,1)
correct_predictions = tf.equal(predictions,true_labels)
p = sess.run(correct_predictions,feed_dict = {x:mnist.train.images,y:mnist.train.labels})

In [30]:
p.sum()

54848